In [1]:
import sys
import pandas as pd
sys.path.append('..')
from configure.settings import DBSelector

In [42]:
db = DBSelector().mongo('qq')['db_stock']
old_doc = db['xueqiu_private']
new_doc =db['xueqiu_private_2021-12-28_brute_force']

In [43]:
only_part_doc =db['xueqiu_private_2021-12-29']

In [44]:
def mongoToDataframe(doc):
    result_list=[]
    for i in doc.find({},{'_id':0}):
        result_list.append(i)

    return pd.DataFrame(result_list)

In [45]:
only_part_df = mongoToDataframe(only_part_doc)

In [46]:
new_df = mongoToDataframe(new_doc)

In [ ]:
old_df.head()

In [47]:
new_df.head()

,symbol,name,profit_rate,annual_return_this_year,manager_nick_name,netvalue_date,create_date,close_status,crwaltime
0,P000048,None,NaN,NaN,None,None,None,False,2021-12-28 22:33:53.371
1,P000049,None,NaN,NaN,None,None,None,False,2021-12-28 22:33:53.710
2,P000051,私募工场(裕泉二期),-14.18,NaN,None,2015-10-27,2014-04-30,True,2021-12-28 22:33:54.102
3,P000057,iMeigu Fund,132.34,-19.24,梁剑,2021-09-30,2013-10-10,True,2021-12-28 22:33:54.689
4,P000090,私募工场(兴富进取1期),81.63,NaN,广东兴富投资,2015-11-06,2014-07-08,True,2021-12-28 22:33:55.562


In [51]:
for index,row in new_df.iterrows():
    symbol = row['symbol']
    if any(only_part_df['symbol']==symbol):
    
        max_drawdown_rate=only_part_df[only_part_df['symbol']==symbol]['max_drawdown_rate'].iloc[0]
        annual_return_year=only_part_df[only_part_df['symbol']==symbol]['annual_return_year'].iloc[0]
        # row['max_drawdown_rate']=max_drawdown_rate
        # row['annual_return_year']=annual_return_year
        new_df.loc[index,'max_drawdown_rate']=max_drawdown_rate
        new_df.loc[index,'annual_return_year']=annual_return_year



In [38]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770 entries, 0 to 769
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   symbol                   770 non-null    object        
 1   name                     716 non-null    object        
 2   profit_rate              677 non-null    float64       
 3   annual_return_this_year  529 non-null    float64       
 4   manager_nick_name        454 non-null    object        
 5   netvalue_date            716 non-null    object        
 6   create_date              716 non-null    object        
 7   close_status             770 non-null    bool          
 8   crwaltime                770 non-null    datetime64[ns]
 9   max_drawdown_rate        397 non-null    float64       
 10  annual_return_year       397 non-null    float64       
dtypes: bool(1), datetime64[ns](1), float64(4), object(5)
memory usage: 61.0+ KB


In [49]:
def convertor(df):
    # df['annual_return_this_year']=df['annual_return_this_year']*100
    df['annual_return_year']=df['annual_return_year']*100
    df['max_drawdown_rate']=df['max_drawdown_rate']*100
    # df['profit_rate']=df['profit_rate']*100
    return df

In [52]:
new_df = convertor(new_df)

In [54]:
process_df = new_df[~new_df['profit_rate'].isnull()]

In [57]:
process_df = process_df[~process_df['name'].isnull()]

In [59]:
len(process_df)

677

In [53]:
new_df.sort_values(by='profit_rate',ascending=False).head(10)

,symbol,name,profit_rate,annual_return_this_year,manager_nick_name,netvalue_date,create_date,close_status,crwaltime,max_drawdown_rate,annual_return_year
515,P000777,大禾投资-掘金5号,1217.20,-6.37,大禾投资,2021-12-24,2016-10-12,False,2021-12-28 22:40:03.485,39.31,-0.30
767,P001029,仙人掌盈沣远航一号,1085.60,98.43,上海仙人掌,2021-12-24,2017-04-12,False,2021-12-28 22:42:14.395,27.97,97.76
684,P000946,正圆壹号,984.00,164.31,正圆投资,2021-12-24,2016-07-14,False,2021-12-28 22:41:37.622,36.38,143.14
691,P000953,涌津涌赢1号,854.70,45.40,涌津投资,2021-12-24,2016-04-28,False,2021-12-28 22:41:43.728,25.21,40.52
603,P000865,达理1号,799.52,17.94,成于思,2021-12-24,2016-07-28,False,2021-12-28 22:40:52.152,34.04,22.67
475,P000736,巨石鑫鼎一号,631.22,175.99,梁雄登,2021-12-24,2020-06-17,False,2021-12-28 22:39:39.807,30.99,184.67
284,P000531,希瓦小牛1号,629.60,1.54,梁宏,2021-12-24,2015-04-21,False,2021-12-28 22:37:01.549,29.79,5.65
690,P000952,龙航一期,611.60,16.29,龙航资产,2021-12-24,2018-12-28,False,2021-12-28 22:41:43.273,26.68,16.29
619,P000881,山楂树一期基金A,526.00,31.40,山楂树甄琢资产,2021-12-24,2014-08-26,False,2021-12-28 22:41:04.147,41.60,33.33
689,P000951,健顺云2号,472.05,24.70,健顺投资,2021-12-10,2015-06-09,False,2021-12-28 22:41:42.820,26.82,40.70


In [9]:
new_df.head()

,symbol,name,profit_rate,annual_return_this_year,manager_nick_name,netvalue_date,create_date,close_status,crwaltime
0,P000048,None,NaN,NaN,None,None,None,False,2021-12-28 22:33:53.371
1,P000049,None,NaN,NaN,None,None,None,False,2021-12-28 22:33:53.710
2,P000051,私募工场(裕泉二期),-14.18,NaN,None,2015-10-27,2014-04-30,True,2021-12-28 22:33:54.102
3,P000057,iMeigu Fund,132.34,-19.24,梁剑,2021-09-30,2013-10-10,True,2021-12-28 22:33:54.689
4,P000090,私募工场(兴富进取1期),81.63,NaN,广东兴富投资,2015-11-06,2014-07-08,True,2021-12-28 22:33:55.562


In [31]:
dead_private_fund = new_df[new_df['netvalue_date']<'2021-11-15']

In [ ]:
dead_private_fund

In [ ]:
old_df = convertor(old_df)

In [34]:
old_symbol_list = old_df['symbol'].values

In [35]:
len(old_symbol_list)

311

In [36]:
new_symbol_list = new_df['symbol'].values

In [37]:
len(new_symbol_list)

420

In [38]:
dead_list = []
for i in old_symbol_list:
    if i not in new_symbol_list:
        dead_list.append(i)

In [39]:
len(dead_list)

64

In [ ]:
dead_list

In [ ]:
new_df.sort_values(by='symbol',ascending=False)['symbol'][:50]

In [5]:
len(new_df)

0

In [14]:
new_df=new_df[new_df['name'].notnull()]

In [15]:
len(new_df)

716

In [19]:
len(new_df[new_df['close_status']==True]) # 关停

78

In [22]:
len(new_df[(new_df['profit_rate']>=0)&(new_df['close_status']==False)])

495

In [68]:
len(process_df)

677

In [67]:
sum(process_df['close_status']==True)

75

In [87]:
closed=process_df[process_df['close_status']==True]

In [ ]:
process_df[process_df['close_status']==True]['profit_rate']

In [66]:
process_df.to_excel("snoball_private_fund.xlsx",encoding="utf8")

In [69]:
75/677

0.11078286558345643

In [89]:
closed['profit_rate'].argmin()

6

In [90]:
closed.iloc[6]

symbol                                        P000211
name                                               赤升
profit_rate                                    -97.49
annual_return_this_year                           NaN
manager_nick_name                                None
netvalue_date                              2018-05-21
create_date                                2015-12-02
close_status                                     True
crwaltime                  2021-12-28 22:34:23.791000
max_drawdown_rate                                 NaN
annual_return_year                                NaN
Name: 15, dtype: object

In [78]:
process_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 677 entries, 2 to 769
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   symbol                   677 non-null    object        
 1   name                     677 non-null    object        
 2   profit_rate              677 non-null    float64       
 3   annual_return_this_year  529 non-null    float64       
 4   manager_nick_name        448 non-null    object        
 5   netvalue_date            677 non-null    object        
 6   create_date              677 non-null    object        
 7   close_status             677 non-null    bool          
 8   crwaltime                677 non-null    datetime64[ns]
 9   max_drawdown_rate        397 non-null    float64       
 10  annual_return_year       397 non-null    float64       
dtypes: bool(1), datetime64[ns](1), float64(4), object(5)
memory usage: 58.8+ KB


In [97]:
import datetime
def convert_year(x):
    try:
        year= (datetime.datetime.strptime(x['netvalue_date'],'%Y-%m-%d') - datetime.datetime.strptime(x['create_date'],'%Y-%m-%d')).days/365
    except:
        year=None
    return year

In [98]:
process_df['duration']=process_df.apply(convert_year,axis=1)

In [100]:
process_df['duration'].argmax()

449

In [101]:
process_df.iloc[449]

symbol                                        P000793
name                            中信信托•神农1期证券投资集合资金信托计划
profit_rate                                    329.26
annual_return_this_year                           NaN
manager_nick_name                             陈营长极品投资
netvalue_date                              2020-10-30
create_date                                2010-07-28
close_status                                    False
crwaltime                  2021-12-28 22:40:14.451000
max_drawdown_rate                                 NaN
annual_return_year                                NaN
duration                                    10.265753
Name: 531, dtype: object

In [102]:
process_df['year_rate'] = process_df['profit_rate']/process_df['duration']

In [106]:
process_df['year_rate'].argmin()

256

In [108]:
process_df.sort_values('year_rate',ascending=True).head(20)

,symbol,name,profit_rate,annual_return_this_year,manager_nick_name,netvalue_date,create_date,close_status,crwaltime,max_drawdown_rate,annual_return_year,duration,year_rate
316,P000564,彼岸成长,-31.30,NaN,None,2018-12-07,2018-08-23,True,2021-12-28 22:38:16.364,NaN,NaN,0.290411,-107.778302
189,P000418,泽道复合策略一期,-42.50,NaN,None,2018-06-22,2017-09-19,True,2021-12-28 22:36:04.322,NaN,NaN,0.756164,-56.204710
249,P000495,南山稳健一号,-33.23,NaN,None,2018-10-19,2018-01-26,True,2021-12-28 22:36:42.378,NaN,NaN,0.728767,-45.597556
394,P000652,测试母基金,-66.33,NaN,None,2019-11-22,2018-05-02,False,2021-12-28 22:38:59.189,NaN,NaN,1.558904,-42.549121
224,P000463,瞭望塔成长,-35.20,NaN,None,2018-10-12,2017-12-04,True,2021-12-28 22:36:25.204,NaN,NaN,0.854795,-41.179487
730,P000992,上海斯诺波嘉源成长,-4.37,-4.37,一手大股东_成嘉源,2021-12-24,2021-11-15,False,2021-12-28 22:41:58.986,5.44,10000.00,0.106849,-40.898718
570,P000832,上海斯诺波领军1号A,-32.10,-32.10,None,2021-12-24,2021-03-09,True,2021-12-28 22:40:32.849,NaN,NaN,0.794521,-40.401724
701,P000963,上海斯诺波领军1号B,-31.40,-31.40,None,2021-12-24,2021-03-09,True,2021-12-28 22:41:47.407,NaN,NaN,0.794521,-39.520690
15,P000211,赤升,-97.49,NaN,None,2018-05-21,2015-12-02,True,2021-12-28 22:34:23.791,NaN,NaN,2.468493,-39.493729
227,P000467,夏雨,-25.50,NaN,None,2018-11-02,2018-02-27,True,2021-12-28 22:36:26.996,NaN,NaN,0.679452,-37.530242
